Links:  
https://towardsdatascience.com/how-tracking-apps-analyse-your-gps-data-a-hands-on-tutorial-in-python-756d4db6715d  



In [1]:
from pathlib import Path
from tqdm import tqdm

import gpxpy
import mplleaflet

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
DATA_PATH = Path('../data')
GPX_DATA = DATA_PATH / 'gpx'

In [12]:
points_list = []

for gpx_file in list(GPX_DATA.glob('*.gpx')):
    with open(gpx_file, 'r') as f:
        gpx = gpxpy.parse(f)
        
    data = gpx.tracks[0].segments[0].points
    
    for point in data:
        points_list.append({'file_name': gpx_file.name, 
                            'lon': point.longitude, 'lat' : point.latitude, 
                            'alt' : point.elevation, 'time' : point.time})
        
df = pd.DataFrame(points_list)
print(df.shape)
df.head()

(303255, 5)


,file_name,lon,lat,alt,time
0,3310079709.gpx,38.123367,55.591830,131.5,2020-02-04 10:10:35+00:00
1,3310079709.gpx,38.123469,55.591913,131.4,2020-02-04 10:10:42+00:00
2,3310079709.gpx,38.123633,55.591948,131.4,2020-02-04 10:10:46+00:00
3,3310079709.gpx,38.123776,55.591903,131.3,2020-02-04 10:10:50+00:00
4,3310079709.gpx,38.123870,55.591801,131.3,2020-02-04 10:10:54+00:00


In [13]:
excluded_files = [
    '3749482548.gpx' # Lake
]

plot_df = df.sort_values('time').copy()
plot_files = plot_df['file_name'].drop_duplicates().tolist()
plot_files = [file_name for file_name in plot_files if file_name not in excluded_files]
plot_df = plot_df[plot_df['file_name'].isin(plot_files)]
print(plot_df.shape)

(302248, 5)


In [14]:
fig, axes = plt.subplots(figsize=(16, 16))

for i, gpx_file in enumerate(plot_files):
    route_df = plot_df[plot_df['file_name'] == gpx_file]
    plt.plot(route_df['lon'], route_df['lat'], linewidth=3)

plt.tight_layout()
mplleaflet.show(fig=fig, path=DATA_PATH/'map.html')